# Text file parsing for iRNA Screening

__Setup__

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import shutil

## iRNA Screening

<img src="./_img/20180803_143938.jpg" width='75%'>

<img src="./_img/image002.png">

## Task

The method of siRNA screening focuses on _switching off_ particular genes in a cell body.
The template comes in a _384 well format_. A robitic instrument with 96 needles does the measurments, hence, during measurment the result is transformed to a 4 x 96 well format. Each of those measurements is then stored in a `.txt` file.

The file names look like this:

```
20180611_112154 NOREAD_PLATE.TXT
20180611_113634 NOREAD_PLATE.TXT
20180611_115108 NOREAD_PLATE.TXT
20180611_120542 NOREAD_PLATE.TXT
20180611_122017 NOREAD_PLATE.TXT
…
```
> __1. The first task is to rename the filnames each corresponding to a block of for files e.g.  `Drugs_A1.1.txt`, `Drugs_A1.2.txt`, ..., `Drugs_A2.1.txt`, ... Note that the files encode the date and time in their filenames.__

The example above would result in something like that:
```
20180611_112154 NOREAD_PLATE.TXT —> Drugs_A1.1.txt
20180611_113634 NOREAD_PLATE.TXT —> Drugs_A1.2.txt
20180611_115108 NOREAD_PLATE.TXT —> Drugs_A1.3.txt
20180611_120542 NOREAD_PLATE.TXT —> Drugs_A1.4.txt
```

> __2. The second task is to extract a particular section of the textifles and store this section in a `.xlsx` file.__

```
Drugs_A1.1.txt —> Drugs_A1.1.xlsx
```



 
 

## Example data

The data is strored in `../data/iRNA_Screening/`

Let us take a loot at one of them:

In [2]:
! cat ../data/iRNA_Screening/20180611_112154\ NOREAD_PLATE.TXT

 �# # B L O C K S =   3 
 N o t e :   
 B a s i c   E n d p o i n t   P r o t o c o l 
 
 U s e   t h i s   p r o t o c o l   f o r   e n d p o i n t   a s s a y s   t h a t   h a v e   u n k n o w n s   t h a t   w i l l   h a v e   c o n c e n t r a t i o n s   i n t e r p o l a t e d   f r o m   a   s t a n d a r d   c u r v e .     M o d i f y   t h e   i n s t r u m e n t   s e t u p   f o r   t h e   w a v e l e n g t h ( s )   o f   i n t e r e s t   f o r   y o u r   a s s a y .     Y o u   m a y   a l s o   m o d i f y   t h e   t e m p l a t e   t o   i n c l u d e   a d d i t i o n a l   s t a n d a r d s ,   u n k n o w n s ,   a n d   c o n t r o l s .     T o   m a k e   m o d i f i c a t i o n s ,   c l i c k   t h e   p l a t e   s e c t i o n   t o   m a k e   i t   a c t i v e . 
 
 R E A D E R   S U I T A B I L I T Y : 
 S p e c t r a M a x   M 2 ,   M 2 e ,   M 3 ,   M 4 ,   M 5 ,   a n d   M 5 e . 
 S p e c t r a M a x   P l u s   3 8 4 ,   1 9 0 ,   S p e c t r a 

## Task 1: Sorting files based on date and time

### Getting the paths

In [3]:
path =  Path('../data/iRNA_Screening/')

In [4]:
path.glob('**/20*.TXT')

<generator object Path.glob at 0x103015048>

In [5]:
list(path.glob('**/20*.TXT'))

[PosixPath('../data/iRNA_Screening/20180611_122017 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_113634 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_115108 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_130359 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_120542 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_140137 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_123450 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_133304 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_134737 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_124923 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_131831 NOREAD_PLATE.TXT'),
 PosixPath('../data/iRNA_Screening/20180611_112154 NOREAD_PLATE.TXT')]

In [6]:
f = list(path.glob('**/20*.TXT'))[0]

In [7]:
f

PosixPath('../data/iRNA_Screening/20180611_122017 NOREAD_PLATE.TXT')

### Getting the date and time from the filename

In [8]:
raw_datetime = f.stem.split(' ')[0]
raw_datetime

'20180611_122017'

In [9]:
datetime = pd.to_datetime(raw_datetime, format='%Y%m%d_%H%M%S')
datetime

Timestamp('2018-06-11 12:20:17')

## Apply logic to all files

In [10]:
out = {}
for f in path.glob('**/20*.TXT'):
    raw_datetime = f.stem.split(' ')[0]
    datetime = pd.to_datetime(raw_datetime, format='%Y%m%d_%H%M%S')
    out[datetime] = f

In [11]:
out

{Timestamp('2018-06-11 12:20:17'): PosixPath('../data/iRNA_Screening/20180611_122017 NOREAD_PLATE.TXT'),
 Timestamp('2018-06-11 11:36:34'): PosixPath('../data/iRNA_Screening/20180611_113634 NOREAD_PLATE.TXT'),
 Timestamp('2018-06-11 11:51:08'): PosixPath('../data/iRNA_Screening/20180611_115108 NOREAD_PLATE.TXT'),
 Timestamp('2018-06-11 13:03:59'): PosixPath('../data/iRNA_Screening/20180611_130359 NOREAD_PLATE.TXT'),
 Timestamp('2018-06-11 12:05:42'): PosixPath('../data/iRNA_Screening/20180611_120542 NOREAD_PLATE.TXT'),
 Timestamp('2018-06-11 14:01:37'): PosixPath('../data/iRNA_Screening/20180611_140137 NOREAD_PLATE.TXT'),
 Timestamp('2018-06-11 12:34:50'): PosixPath('../data/iRNA_Screening/20180611_123450 NOREAD_PLATE.TXT'),
 Timestamp('2018-06-11 13:33:04'): PosixPath('../data/iRNA_Screening/20180611_133304 NOREAD_PLATE.TXT'),
 Timestamp('2018-06-11 13:47:37'): PosixPath('../data/iRNA_Screening/20180611_134737 NOREAD_PLATE.TXT'),
 Timestamp('2018-06-11 12:49:23'): PosixPath('../data/i

### Group files based on datetime and rename

In [12]:
for k in sorted(out.keys()):
    print(k)

2018-06-11 11:21:54
2018-06-11 11:36:34
2018-06-11 11:51:08
2018-06-11 12:05:42
2018-06-11 12:20:17
2018-06-11 12:34:50
2018-06-11 12:49:23
2018-06-11 13:03:59
2018-06-11 13:18:31
2018-06-11 13:33:04
2018-06-11 13:47:37
2018-06-11 14:01:37


In [13]:
index = list(range(1,int(np.ceil((len(out.keys())+1)/4))))
index

[1, 2, 3]

In [14]:
# set starting values
index_pos = -1 # -1 instead of 0 as 0 % 4 == 0!
sub_index = 1
txt = 'Drugs_A'

for e, k in enumerate(sorted(out.keys())):
    if e % 4 == 0:
        # if e is 0, 4, 8, 12, ... do:
        index_pos += 1
        sub_index = 1
        print (f'\nIndex position = {index_pos}')
    
    # generate new filename
    new_name = f'{txt}{index[index_pos]}.{sub_index}.TXT'
    print(f'Old name: {k} --> New name: {new_name}')
    sub_index += 1
    
    # copy file with new name
    source_name = out[k]
    shutil.copyfile(source_name, path.joinpath(new_name))
    


Index position = 0
Old name: 2018-06-11 11:21:54 --> New name: Drugs_A1.1.TXT
Old name: 2018-06-11 11:36:34 --> New name: Drugs_A1.2.TXT
Old name: 2018-06-11 11:51:08 --> New name: Drugs_A1.3.TXT
Old name: 2018-06-11 12:05:42 --> New name: Drugs_A1.4.TXT

Index position = 1
Old name: 2018-06-11 12:20:17 --> New name: Drugs_A2.1.TXT
Old name: 2018-06-11 12:34:50 --> New name: Drugs_A2.2.TXT
Old name: 2018-06-11 12:49:23 --> New name: Drugs_A2.3.TXT
Old name: 2018-06-11 13:03:59 --> New name: Drugs_A2.4.TXT

Index position = 2
Old name: 2018-06-11 13:18:31 --> New name: Drugs_A3.1.TXT
Old name: 2018-06-11 13:33:04 --> New name: Drugs_A3.2.TXT
Old name: 2018-06-11 13:47:37 --> New name: Drugs_A3.3.TXT
Old name: 2018-06-11 14:01:37 --> New name: Drugs_A3.4.TXT


## Task 2: Extract a particular section of the textifle and store this section in a `.xlsx` file.

In [15]:
#! cat ../data/iRNA_Screening/Drugs_A1.1.TXT

In [16]:
#pd.read_csv('../data/iRNA_Screening/Drugs_A1.1.TXT')

### Load data using pandas powerful `read_csv` function

In [17]:
df_raw = pd.read_csv('../data/iRNA_Screening/Drugs_A1.1.TXT', 
                     skiprows=55, 
                     skipfooter=79, 
                     sep='\t', 
                     encoding='utf-16', 
                     engine='python')
df_raw

,Unnamed: 0,Unnamed: 1,,.1,.2,.3,.4,.5,.6,.7,.8,.9,.10,.11,Unnamed: 14
0,NaN,NaN,8567680.75,6146100.25,7947892.50,7730412.75,7330878.50,6948680.50,6530143.00,6194062.25,6137654.75,7906526.50,6459051.25,7789278.00,NaN
1,NaN,NaN,8055890.25,7562240.00,6275937.50,6389713.75,6645504.50,5905482.75,6994168.50,6647997.25,6913279.00,7639214.25,7203998.00,7657190.00,NaN
2,NaN,NaN,7592615.00,6594851.00,5016483.50,7348367.75,6305780.25,6318335.25,6850932.50,6659728.50,6312526.25,6418770.00,6250303.25,7943954.50,NaN
3,NaN,NaN,8150232.50,7066456.25,7760066.75,7129537.00,7201987.00,8308010.75,6641123.00,7573502.50,5927940.75,7818307.00,6369931.75,7857027.25,NaN
4,NaN,NaN,7478538.00,10230818.50,7886834.00,6965749.25,8123084.00,7204249.75,6547623.75,7324734.75,6608158.50,5664928.25,6398675.00,7739291.25,NaN
5,NaN,NaN,7791921.25,6690641.00,9898784.00,6392149.25,7731684.25,5927139.00,7037831.25,6885562.25,8139760.50,8085957.50,8239345.75,7834852.50,NaN


### Subset the data

In [18]:
df = df_raw[df_raw.columns[2:14]]
df.columns = range(1,13)
df

,1,2,3,4,5,6,7,8,9,10,11,12
0,8567680.75,6146100.25,7947892.50,7730412.75,7330878.50,6948680.50,6530143.00,6194062.25,6137654.75,7906526.50,6459051.25,7789278.00
1,8055890.25,7562240.00,6275937.50,6389713.75,6645504.50,5905482.75,6994168.50,6647997.25,6913279.00,7639214.25,7203998.00,7657190.00
2,7592615.00,6594851.00,5016483.50,7348367.75,6305780.25,6318335.25,6850932.50,6659728.50,6312526.25,6418770.00,6250303.25,7943954.50
3,8150232.50,7066456.25,7760066.75,7129537.00,7201987.00,8308010.75,6641123.00,7573502.50,5927940.75,7818307.00,6369931.75,7857027.25
4,7478538.00,10230818.50,7886834.00,6965749.25,8123084.00,7204249.75,6547623.75,7324734.75,6608158.50,5664928.25,6398675.00,7739291.25
5,7791921.25,6690641.00,9898784.00,6392149.25,7731684.25,5927139.00,7037831.25,6885562.25,8139760.50,8085957.50,8239345.75,7834852.50


### Apply logic to all files and safe them as `.xlsx` files

In [19]:
list(path.glob('**/Drug*.TXT'))

[PosixPath('../data/iRNA_Screening/Drugs_A1.4.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A3.4.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A1.2.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A1.3.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A3.1.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A3.3.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A1.1.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A3.2.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A2.4.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A2.3.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A2.2.TXT'),
 PosixPath('../data/iRNA_Screening/Drugs_A2.1.TXT')]

In [20]:
for f in path.glob('**/Drug*.TXT'):
    # load the data
    df_raw = pd.read_csv(f, 
                         skiprows=55, 
                         skipfooter=79, 
                         sep='\t', 
                         encoding='utf-16', 
                         engine='python')
    # subset
    df = df_raw[df_raw.columns[2:14]]
    df.columns = range(1,13)
    # save
    filename = Path(*f.parts[:-1]).joinpath(f.parts[-1].replace('TXT', 'xlsx'))
    df.to_excel(filename, index=False)
    

***